In [ ]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [1]:
from openai import OpenAI
import json

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location."""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": 10, "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": 72, "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": 22, "unit": unit})
    else:
        raise ValueError(f"Unknown {location}!")

In [3]:
openai = OpenAI()

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g., San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [5]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in San Francisco, Tokyo, and Paris?",
    }
]

In [6]:
# type: ignore
resp = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125", messages=messages, tools=tools, tool_choice="auto"
)

In [7]:
resp_message = resp.choices[0].message
resp_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NchEAkgwvVGFicoN8oCU9vVW', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_2TvFz9ILZWj0dMtcbcmGVJxH', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_Zhmquae1qK8CPzoUUl40RsI3', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')])

In [8]:
messages.append(resp_message)  # type: ignore

In [9]:
funcs = {"get_current_weather": get_current_weather}

In [10]:
# type: ignore
for tool_call in resp_message.tool_calls:
    if tool_call.type == "function":
        func = funcs[tool_call.function.name]
        kwargs = json.loads(tool_call.function.arguments)
        output = func(**kwargs)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": output,
            }
        )

In [11]:
messages

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NchEAkgwvVGFicoN8oCU9vVW', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_2TvFz9ILZWj0dMtcbcmGVJxH', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_Zhmquae1qK8CPzoUUl40RsI3', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_NchEAkgwvVGFicoN8oCU9vVW',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "San Francisco", "temperature": 72, "unit": "celsius"}'},
 {'tool_call_id': 'call_2TvFz9ILZWj0dMtc

In [12]:
# type: ignore
resp = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125", messages=messages, tools=tools, tool_choice="auto"
)

In [13]:
print(resp.choices[0].message.content)

The current weather in:
- San Francisco is 72°C
- Tokyo is 10°C
- Paris is 22°C
